In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
cost_folder = r"" #path to specific customer ap folder
receivable_folder = r""#path to specific customer ar folder

def convert_waybill_number(value):
    if isinstance(value, (int, float)):
        return str(int(value)) + '\t'
    else:
        return str(value) + '\t'

chitacost = []
for root, dirs, files in os.walk(cost_folder):
    for file in files:
        if file.endswith('.xlsx'):
            file_path = os.path.join(root, file)
            print(file_path)
            df = pd.read_excel(file_path)
            df.fillna(0, inplace=True)
            df['Express or Ground Tracking ID'] = df['Express or Ground Tracking ID'].apply(convert_waybill_number)
            chitacost.extend(df['Express or Ground Tracking ID'].tolist())

chita_receivable = []
for root, dirs, files in os.walk(receivable_folder):
    print(root)
    for file in files:
        if file.endswith('.csv'):
            file_path = os.path.join(root, file)
            print(file_path)
            df = pd.read_csv(file_path)
            df.fillna(0, inplace=True)
            df['Waybill Number'] = df['Waybill Number'].apply(convert_waybill_number)
            chita_receivable.extend(df['Waybill Number'].tolist())

common = list(set(chitacost) & set(chita_receivable))

extra_in_receivable = list(set(chita_receivable) - set(common))

print(len(extra_in_receivable))

In [ ]:
print(extra_in_receivable)

In [ ]:
max_len = max(len(chitacost), len(chita_receivable))
chita_receivable.extend([np.nan] * (max_len - len(chita_receivable)))
common.extend([np.nan] * (max_len - len(common)))
extra_in_receivable.extend([np.nan] * (max_len - len(extra_in_receivable)))


print(max_len)
print(len(chitacost))
print(len(chita_receivable))
print(len(common))
print(len(extra_in_receivable))

output_data = {'chitacost': chitacost, 'chita_receivable': chita_receivable, 'common': common, 'extra': extra_in_receivable}
output_df = pd.DataFrame(output_data)
output_df.to_excel(r"", index=False) #path to output file


In [ ]:
common_data_a = pd.DataFrame()
common_data_l = pd.DataFrame()
extra_data_a = pd.DataFrame()
extra_data_l = pd.DataFrame()

for root, dirs, files in os.walk(cost_folder):
    for file in files:
        if file.endswith('.xlsx'):
            file_path = os.path.join(root, file)
            print(file_path)
            df = pd.read_excel(file_path)
            df.fillna(0, inplace=True)
            df['Express or Ground Tracking ID'] = df['Express or Ground Tracking ID'].apply(convert_waybill_number)
            if '大方广' in file:
                print(file)
                common_data_a = pd.concat([common_data_a, df[df['Express or Ground Tracking ID'].isin(common)]])
                extra_data_a = pd.concat([extra_data_a, df[df['Express or Ground Tracking ID'].isin(extra_in_receivable)]])
            else:
                print(file)
                common_data_l = pd.concat([common_data_l, df[df['Express or Ground Tracking ID'].isin(common)]])
                extra_data_l = pd.concat([extra_data_l, df[df['Express or Ground Tracking ID'].isin(extra_in_receivable)]])

with pd.ExcelWriter(r"", engine='openpyxl', mode='a') as writer: #path to output file
    common_data_a.to_excel(writer, sheet_name="重叠的成本账单FedexA", index=False)
    common_data_l.to_excel(writer, sheet_name="重叠的成本账单FedexL", index=False)
    extra_data_a.to_excel(writer, sheet_name="不重叠的FedexA", index=False)
    extra_data_l.to_excel(writer, sheet_name="不重叠的FedexL", index=False)

